In [1]:
!pip install -U transformers accelerate datasets bitsandbytes peft trl datasets
!pip install --upgrade protobuf==4.21.5
!pip install sentencepiece


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments,BitsAndBytesConfig,pipeline,Trainer,DataCollatorForLanguageModeling
import torch
from peft import LoraConfig,PeftModel,prepare_model_for_kbit_training,get_peft_model,TaskType
from trl import SFTTrainer
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
dataset = load_dataset("b-mc2/sql-create-context")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [5]:

base_model_id= "NousResearch/Hermes-2-Pro-Mistral-7B"
#tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False)
tokenizer= AutoTokenizer.from_pretrained(base_model_id,use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

In [6]:
def tokenize_data(sample):
    start_prompt= "Write SQL query according to given question or instruction.\n\n"
    end_prompt= "\n\nResponse:"
    prompt= [start_prompt+question+end_prompt for question in sample['question']]
    sample['input_ids']= tokenizer(prompt,padding='max_length',truncation=True,return_tensors='pt',max_length=512,).input_ids
    sample['labels']= tokenizer(sample['answer'],padding='max_length',truncation=True,return_tensors='pt',max_length=512).input_ids
    
    return sample

In [7]:
dataset= dataset.map(tokenize_data,batched=True)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [8]:
dataset=dataset.remove_columns(['answer', 'question', 'context'])

In [9]:
dataset=dataset['train']

In [10]:
dataset= dataset.select(range(1000))

In [15]:
# Quantization Config 
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True)

# LoRA COnfig
peft_config= LoraConfig(lora_alpha=15,
                        lora_dropout=0.1,
                        target_modules=[
                                    'q_proj',
                                    'k_proj',
                                    'v_proj',
                                    'dense'],
                        bias="none",
                        task_type=TaskType.CAUSAL_LM)

In [16]:
base_model= AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=bnb_config,device_map="auto")
base_model.config.use_cache=False

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
quatnized_model= prepare_model_for_kbit_training(base_model)

In [18]:
peft_model= get_peft_model(model=quatnized_model,peft_config=peft_config)

In [19]:
merged_model = peft_model.merge_and_unload()

In [20]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
training_args= TrainingArguments(
    output_dir='./working/Mistral-7B-Fine_Tunned',
    hub_model_id='dbuddenbaum/Mistral-7B-Fine_Tunned',
    learning_rate=2e-4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_steps=10,
    save_strategy='epoch',
    save_steps=10)

trainer= Trainer(
    model= peft_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,10.140200,10.109017


TrainOutput(global_step=125, training_loss=10.111825653076172, metrics={'train_runtime': 1501.9306, 'train_samples_per_second': 0.666, 'train_steps_per_second': 0.083, 'total_flos': 2.1844350468096e+16, 'train_loss': 10.111825653076172, 'epoch': 1.0})

In [30]:
new_model= "Mistral-7B-Fine_Tunned1"

In [31]:
trainer.model.save_pretrained(new_model)

In [32]:
new_model= './working/Mistral-7B-Fine_Tunned1'

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Define the base model ID or path
base_model = 'NousResearch/Hermes-2-Pro-Mistral-7B'  # replace with actual path or model ID

# Load the base model with appropriate parameters
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             low_cpu_mem_usage=True,
                                             return_dict=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")

# Define the path or model ID for the QLoRA adapters
lora_adapters = './Mistral-7B-Fine_Tunned1'  # replace with actual path or model ID

# Load the QLoRA adapters into the model
model = PeftModel.from_pretrained(model, lora_adapters)

# Merge the QLoRA adapters into the base model and unload adapters
model = model.merge_and_unload()

# Load the tokenizer for the base model
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Some parameters are on the meta device because they were offloaded to the cpu.


In [36]:
model.push_to_hub("Mistral-7B-Fine_Tunned_Model")

Saving checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/9.54G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dbuddenbaum/Mistral-7B-Fine_Tunned_Model/commit/f84aae339061d0c5dab4e8f61d5163eb6129b750', commit_message='Upload MistralForCausalLM', commit_description='', oid='f84aae339061d0c5dab4e8f61d5163eb6129b750', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dbuddenbaum/Mistral-7B-Fine_Tunned_Model', endpoint='https://huggingface.co', repo_type='model', repo_id='dbuddenbaum/Mistral-7B-Fine_Tunned_Model'), pr_revision=None, pr_num=None)

In [37]:
tokenizer.push_to_hub("Mistral-7B-Fine_Tunned_Model")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dbuddenbaum/Mistral-7B-Fine_Tunned_Model/commit/e81f43e67e5a6101a95bf93e6cff8f004b4cbdf1', commit_message='Upload tokenizer', commit_description='', oid='e81f43e67e5a6101a95bf93e6cff8f004b4cbdf1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dbuddenbaum/Mistral-7B-Fine_Tunned_Model', endpoint='https://huggingface.co', repo_type='model', repo_id='dbuddenbaum/Mistral-7B-Fine_Tunned_Model'), pr_revision=None, pr_num=None)

In [38]:
ll= '''You have a database with two tables: employees and departments. The employees table contains information about each employee, and the departments table contains information about each department.Write an SQL query to retrieve the names of all employees along with the name of the department they belong to.'''

In [39]:
start_prompt= "Write SQL query according to given question or instruction.\n\n"
end_prompt= "\n\nResponse:"
prompt= start_prompt+ll+end_prompt

pipe= pipeline(task='text-generation',model=model,tokenizer=tokenizer,max_length=200)
result= pipe(prompt)
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Write SQL query according to given question or instruction.

You have a database with two tables: employees and departments. The employees table contains information about each employee, and the departments table contains information about each department.Write an SQL query to retrieve the names of all employees along with the name of the department they belong to.

Response:
SELECT employees.name, departments.name AS department
FROM employees
JOIN departments ON employees.department_id = departments.id;

The SQL query above uses a JOIN statement to combine the employees and departments tables based on the department_id column in the employees table and the id column in the departments table. The SELECT statement then retrieves the name column from both tables, using the AS keyword to alias the departments.name column as department. This will give us the names of all employees and the names of the departments they belong to in the result set.

If you have any further questions
